# DeepFaceLab Colab Version by [Deepartist](mailto:deepartist.ai@gmail.com)
This is based on [DFL-Colab](https://github.com/chervonij/DFL-Colab), [DeepFaceLab-Linux](https://github.com/nagadit/DeepFaceLab_Linux)






# Installation

In [ ]:
#@title Check GPU and CPU  
gpu = !nvidia-smi -L
gpu= gpu[0]
gpu_mem = !nvidia-smi --query-gpu=memory.total --format=csv
gpu_mem = gpu_mem[1]
gpu_mem_free = !nvidia-smi --query-gpu=memory.free --format=csv
gpu_mem_free = gpu_mem_free[1]
cores = !grep -c ^processor /proc/cpuinfo
cores = cores[0]
mhz = !grep "^[c]pu MHz" /proc/cpuinfo
mhz = mhz[0].split(":")[-1]
mhz
!echo "$gpu"
!echo Total Memory: "$gpu_mem" Free: "$gpu_mem_free"
!echo CPU: "$cores" cores @"$mhz" MHz

**Run to mount your Google drive with auth key**

1. Connect to your google drive to be able to import and export files to/from google drive
2. Click on Link below
3. Login and confirm
4. Ctrl+V auth key and hit return key

**OR skip and mount your Google drive permantly**

1. Create new notebook
2. Change runtime to GPU
3. Click on files, and mount your google drive permantly 
4. Switch to this notenook
4. Click in first cell (title)
5. Select all cells with shift+ arrow down
6. Copy all cells from this notebook (ctrl+c)
8. Click in first cell in the new notebook and paste (ctrl+v)
7. Save notebook in drive -> No more auth token coyping

**OR SIMPLY SKIP** if you dont want import and export files to/from google drive

In [ ]:
#@title Connect to your gdrive OR ignrore if permantly mounted
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Enter puplic or private DeepFaceLab Git


gitlink = 'https://github.com/iperov/DeepFaceLab.git' #@param['https://github.com/iperov/DeepFaceLab.git', 'https://github.com/faceshiftlabs/DeepFaceLab.git', 'https://github.com/janfschr/DeepFaceLab_Fork.git']
gitlink_overwrite = "" #@param{type:"string"}
if gitlink_overwrite is not None and gitlink_overwrite.strip() is not "":
  gitlink = gitlink.strip()

#@markdown Enter specific branch , empty is master 
branch = "" #@param {type:"string"}

if branch is not None and branch.strip() is not "":
  gitlink+= " -b "+branch.strip()
#@markdown Check if private, enter user and enter password or enter hidden in prombt 
is_private = False #@param {type:"boolean"}
user = 'janfschr' #@param {type:"string"}
clear_password = 'not_safu' #@param {type:"string"}

if is_private:
  import urllib.parse
  if user is None or user.strip() is "": raise ValueError('Empty username')
  if clear_password is None or clear_password.strip() =='':
    from getpass import getpass
    password = getpass('Enter your password').strip()
    password = urllib.parse.quote(password)
  else: password = clear_password.strip()
  gitlink = gitlink.replace("https://", "")
  hidden = "******"
  print(f"https://{user.strip()}:{password if clear_password else hidden}@"+gitlink)
  gitlink = f"https://{user.strip()}:{password}@"+gitlink
else: 
  print(gitlink)

!echo "$gitlink" > /content/gitlink

In [ ]:
#@title Install or update DeepFaceLab from Github
%tensorflow_version 2.x

this_version = "1.0.0"

from pathlib import Path
Mode = "forced_reinstall" #@param ["install", "update", "forced_reinstall"]
##@markdown Experimental: Keep tensorflow version of colab 
#keep_installed_tf = True #@param {type:"boolean"}

#@markdown Some packages (e.g. Tensorflow) needs a restart, check if this should happen automatically
auto_restart = True #@param{type:"boolean"}
#@markdown Check if you dont use Quick96 / generic Xseg or want to do a pretraining

skip_pretrain = True #@param {type:"boolean"}
skip_pretrain_q96 = True #@param {type:"boolean"}
skip_generic_xseg = True #@param {type:"boolean"}
#@markdown Experimental: Deleted unneeded files and libs
free_up_diskspace = True #@param {type:"boolean"}



pretrain_link ="https://github.com/janfschr/DeepFaceLab_Colab/releases/download/1.0/pretrain_FFHQ.zip"


install_path = "/content/"
dfl_dir = "DeepFaceLab/"

!echo "$install_path" > /content/install_path
gitlink = !cat /content/gitlink
gitlink = gitlink[0]

if gitlink is None or gitlink.strip() == "" or "cat" in gitlink: 
  raise ValueError("Run cell above for creating gitlink ")

if Mode == "forced_reinstall": 
  !rm -rf $dfl_dir

if "install" in Mode:
  !mkdir -p $install_path
  %cd $install_path
  #!git clone https://github.com/iperov/DeepFaceLab.git
  !git clone $gitlink $dfl_dir
  
  # fix linux warning
  # /usr/lib/python3.6/multiprocessing/semaphore_tracker.py:143: UserWarning: semaphore_tracker: There appear to be 1 leaked semaphores to clean up at shutdown
  fin = open("/usr/lib/python3.6/multiprocessing/semaphore_tracker.py", "rt")
  data = fin.read()
  data = data.replace('if cache:', 'if False:')
  fin.close()

  fin = open("/usr/lib/python3.6/multiprocessing/semaphore_tracker.py", "wt")
  fin.write(data)
  fin.close()
  
  import tensorflow
  req_file = install_path + "DeepFaceLab/requirements-colab.txt"
  # reqs = !cat $req_file 
  # tf = [i for i in reqs if 'tensorflow-gpu' in i]
  # tf_v = tf[0].split("==")[-1]
  # #print (tf_v)
  # print(f"Need tensorflow {tf_v}")
  # if not tf_v in tensorflow.__version__ and not keep_installed_tf:
  #   !pip uninstall -y tensorflow tensorflow-gpu
  #   pass
  !pip install -r $req_file
  !pip install --upgrade scikit-image
  !sudo apt install cuda-10-0
  #configure cron
  !sudo apt install cron -y
  !sudo service cron start
  !sudo service cron stop
  !cp /etc/crontab /etc/crontab.ORG

  #create default workspace
  workspace_path = install_path+"workspace/"
  !echo "$workspace_path" > /content/workspace_path
  if not Path(workspace_path).exists():
    !mkdir -p $workspace_path""data_src/aligned
    !mkdir -p $workspace_path""data_dst/aligned
    !mkdir -p $workspace_path""model


  #install scripts
  %cd /content/
  !git clone https://github.com/janfschr/DeepFaceLab_Colab #TODO use own repo
  git_notebook_version = !cat /content/DeepFaceLab_Colab/version
  git_notebook_version = git_notebook_version[0]
  need_update = git_notebook_version != this_version
  !mv DeepFaceLab_Colab/scripts /content/scripts
  !rm -rf DeepFaceLab_Colab
  #TODO check for update in git 




elif Mode=="update":
  %cd $dfl_dir
  !git pull

  install_path = !cat /content/install_path
  install_path = install_path[0]
  workspace_path = !cat /content/workspace_path
  workspace_path = workspace_path[0]

pretrain_install_path = install_path+"pretrain"
pretrain_q_96_install_path = install_path+"pretrain_Q96"
generic_xseg_install_path = install_path+"genericXseg"

#check if pretrain folder exists
ls = !ls $install_path
if not "pretrain" in ls.nlstr and not skip_pretrain:
  print("Downloading Pretrain faceset ... ")
  !wget -q --no-check-certificate -r $pretrain_link -O /content/pretrain_faceset.zip
  !mkdir $pretrain_install_path
  !unzip -q /content/pretrain_faceset.zip -d $pretrain_install_path
  !rm /content/pretrain_faceset.zip
else:
  print("Folder pretrain already exists in %s "%install_path)

if not "pretrain_Q96" in ls.nlstr and not skip_pretrain_q96:
  print("Downloading Q96 pretrained model ...")
  !wget -q --no-check-certificate -r 'https://github.com/chervonij/DFL-Colab/releases/download/Q96_model_pretrained/Q96_model_pretrained.zip' -O /content/pretrain_Q96.zip
  !mkdir $pretrain_q_96_install_path
  !unzip -q /content/pretrain_Q96.zip -d $pretrain_q_96_install_path
  !rm /content/pretrain_Q96.zip 
else: 
  print("Folder pretrain_Q96 already exists in %s "%install_path)


if not "generic_xseg" in ls.nlstr and not skip_generic_xseg:
  #TODO
  print("Downloading generic pretrained model ...")
  !wget -q --no-check-certificate -r 'https://github.com/janfscher/DFL-Linux/releases/downloadgeneric_xseg.zip' -O /content/genericXxseg.zip
  !mkdir $generic_xseg_install_path
  !unzip -q /content/genericXseg.zip -d $generic_xseg_install_path
  !rm /content/generic_xseg.zip 

from google.colab import output
output.clear()

print("\nDone!")
print("Install path: ")
%cd $install_path

if need_update:
  print("\nOutdated notebook version, get new notebook from https://github.com/JanFschr/DeepFaceLab_Colab")

if free_up_diskspace: #highly experimental!
  print("\nCleaning up some space!")
  !sudo rm -rf /usr/local/lib/python2.7
  !sudo rm -rf /usr/local/lib/python3.8
  !sudo rm -rf /usr/local/lib/python3.6
  # !sudo rm -rf /usr/local/cuda-10.1
  # !sudo rm -rf /usr/local/cuda-10.0
  !sudo rm -rf /tensorflow-1.15.2




if auto_restart and "install" in Mode:
  !echo "Auto Restart: Session will crash now!"
  import time
  time.sleep(1)
  import os
  os.kill(os.getpid(), 9)

#Manage workspace

*   You can upload to colab (zip or folder with faceset.pak) OR
*   use your gdrive directly (zip or folder with faceset.pak)
*   or delete and recreate your workspace and 
*   sync with gdrive 

Dont use gdrive with a lot of single images, its slow and you will exceed your quota





## Up/Download


In [ ]:
#@title Upload zipped workspace
%cd /content/
import os 
from pathlib import Path
required_folders = ["data_dst", "data_src", "model"]
workspace_path = !cat /content/workspace_path
workspace_path = workspace_path[0]
#@markdown Delete workspace on colab drive
delete_workspace = True #@param {type:"boolean"}

from google.colab import files
uploaded = files.upload()
filename = uploaded[0]
!rm -rf /content/tmp
!mkdir -p /content/tmp
!7z e /content/$filename -o /content/tmp
#check if folder contains "workspace"
tmp_ls = !ls /content/tmp
tmp_ls = tmp_ls[0].split(" ")

if delete_workspace:
  !rm -rf $workspace_path

if len(tmp_ls) == 0 and "workspace" in tmp_ls[0]:
  !rsync -ah /content/tmp $workspace_path
  for f in required_folders:
    !rsync -ah "/content/tmp/workspace/"$f $workspace_path
else if not any(f not in " ".join(tmp_ls) for f in required_folders):
  for f in required_folders:
    !rsync -ah "/content/tmp/"$f $workspace_path
else:
  raise ValueError(f"Extracted zip is no workspace)

In [ ]:
#@title Zip and Download your workspace
# !rm -rf /content/tmp
# !mkdir -p /content/tmp
# !7z a -mx0 -tzip  /content/tmp/workspace.zip -w $workspace_path
#!rsync --ah --info=progress2 -rf $workspace_path /content/tmp/
from google.colab import files
files.download(workspace_path)

print(f"Or download manually: {workspace_path}")




##Gdrive 

In [ ]:
#@title Sync workspace from drive 
#@markdown Use zipped or workspace folder with packed faces

#@markdown No need to enter /content/gdrive/MyDrive/ 

#@markdown Enter folder or path containing the workspace folder OR full path zip file
%cd /content/
import os 
from pathlib import Path
required_folders = ["data_dst", "data_src", "model"]
gdrive_root =  "/content/drive/MyDrive/"
workspace_on_gdrive = "deepfakes/workspaces/" #@param {type:"string"}
#@markdown Delete workspace on colab drive
delete_local_workspace = True #@param {type:"boolean"}

if not Path(gdrive_root).exists():
  raise ValueError("Google drive not mounted")

workspace_path = !cat /content/workspace_path
workspace_path = workspace_path[0]
if workspace_path is None or workspace_path.strip() == "" or "cat" in workspace_path: 
  raise ValueError("No workspace set!")

if workspace_on_gdrive.split(".")[-1] == "zip":
  workspace_full = os.path.join(gdrive_root, workspace_on_gdrive)
  !rsync -ah --info=progress2 $workspace_full"/"$f /content
  filename = Path(workspace_full).basename
  !rm -rf /content/tmp
  !mkdir -p /content/tmp
  !7z e /content/$filename -o /content/tmp
  #check if folder contains "workspace"
  tmp_ls = !ls /content/tmp
  tmp_ls = tmp_ls[0].split(" ")

  

  if len(tmp_ls) == 0 and "workspace" in tmp_ls[0]:
    if delete_local_workspace:
      !rm -rf $workspace_path
      !mkdir -p $workspace_path
    !rsync -ah /content/tmp $workspace_path
    for f in required_folders:
      !rsync -ah "/content/tmp/workspace/"$f $workspace_path
  elif not any(f not in " ".join(tmp_ls) for f in required_folders):
    if delete_local_workspace:
      !rm -rf $workspace_path
      !mkdir -p $workspace_path
    for f in required_folders:
      !rsync -ah "/content/tmp/"$f $workspace_path
  else:
    raise ValueError(f"Extracted zip is no workspace")
  pass
else:
  if workspace_on_gdrive[:-1] == "/":
    workspace_on_gdrive=workspace_on_gdrive[:-1]
  workspace_full = os.path.join(gdrive_root, workspace_on_gdrive)
  ls_folder = !ls $workspace_full
  ls_folder = ls_folder[0].split(" ")
  if len(ls_folder) == 0: raise ValueError(f"{workspace_full} empty")
  elif len(ls_folder) > 1 and not any(f not in " ".join(ls_folder) for f in required_folders):
    #workspace_full = Path(workspace_full).parent
    pass
  elif len(ls_folder) == 1 and "workspace" == ls_folder[0]:
    workspace_full = os.path.join(workspace_full, "workspace")
    ls_folder = !ls $workspace_full
    ls_folder = ls_folder[0].split(" ")
    if len(ls_folder) > 1 and not any(f not in " ".join(ls_folder) for f in required_folders):
      pass
    else:
      raise ValueError(f"{workspace_full} is no workspace")
  else:
    raise ValueError(f"{workspace_full} is no workspace")


  if delete_local_workspace:
    !rm -rf $workspace_path
    !mkdir -p $workspace_path
  for f in required_folders:
    print(f"Syncing {workspace_full}/{f}")
    !rsync -ah --exclude {"*_history"} --exclude {"*_autobackups"}  --info=progress2 $workspace_full"/"$f $workspace_path

!echo $workspace_full"/model" > /content/model_path_gdrive

  

In [ ]:
#@title Sync workspace to drive 
#@markdown Use zipped or workspace folder with packed faces

#@markdown No need to enter /content/gdrive/MyDrive/ 

#@markdown Enter folder or path containing the workspace folder OR full path zip file

import os 
gdrive_root =  "/content/drive/MyDrive/"
workspace_on_gdrive = "deepfakes/workspaces/" #@param {type:"string"}
workspace_full = os.path.join(gdrive_root, workspace_on_gdrive
from pathlib import Path
if not Path(gdrive_root).exists():
  raise ValueError("Google drive not mounted")

workspace_path = !cat /content/workspace_path
workspace_path = workspace_path[0]
if workspace_path is None or workspace_path.strip() == "" or "cat" in workspace_path: 
  raise ValueError("No workspace set!")

                              
if zip in workspace_on_gdrive.split(".")[-1]:
  !7z a -tzip $workspace_on_gdrive -w workspace_path
else: 
  !rsync -ah --info=progress2  $workspace_path $workspace_full


#Training

In [ ]:
install_path = !cat /content/install_path
install_path = install_path[0]

model_path = !cat /content/model_path
model_path = model_path[0]


# model_path_gdrive= !cat /content/model_path_gdrive
# model_path_gdrive= model_path_gdrive[0]
# if model_path_gdrive is None or model_path_gdrive.strip() == "" or "cat" in model_path_gdrive:
#   model_path_gdrive = gdrive_root =  "/content/drive/MyDrive/"

workspace_path = !cat /content/workspace_path
workspace_path = workspace_path[0]
if workspace_path is None or workspace_path.strip() == "" or "cat" in workspace_path: 
  raise ValueError("No workspace set!")



#@title Training
Model = "SAEHD" #@param ["SAEHD", "AMP", "Quick96", "XSeg"]
Backup_hourly_on_gdrive = True #@param {type:"boolean"}
Colab_Pro = True #@param {type:"boolean"}

if Colab_Pro:
  max_time = int(23.5 * 60 * 60)
else: 
  max_time = int(11.5 * 60 * 60)

Silent_Start = True #@param {type:"boolean"}

%cd $install_path



import psutil, os, time
from pathlib import Path

gdrive_root =  "/content/drive/MyDrive/"
if not Path(gdrive_root).exists():
  raise ValueError("Google drive not mounted")

model_path = os.path.join(workspace_path, "model")
!mkdir -p $model_path
#@markdown This needs to be a folder !
zip_before_backup = True #@param {type:"boolean"}
workspace_on_gdrive = "deepfakes/workspaces/" #@param {type:"string"}
workspace_full = os.path.join(gdrive_root, workspace_on_gdrive)

p = psutil.Process(os.getpid())
uptime = time.time() - p.create_time()

# if (Backup_every_hour):
#   if not os.path.exists('workspace.zip'):
#     print("Creating workspace archive ...")
#     !zip -r -q workspace.zip workspace
#     print("Archive created!")
#   else:
#     print("Archive exist!")

#create cron rsyncjob 
!sudo service cron stop
!rm -rf /etc/crontab
!cp /etc/crontab.ORG /etc/crontab

if zip_before_backup:
  !echo 'rm -rf /content/tmp; mkdir -p /content/tmp' > /content/cronjob.sh
  !echo '7z a -mx0 -tzip /content/tmp/model.zip $model_path' >> /content/cronjob.sh
  !echo 'bash $install_path"scripts/0_rsync.sh" $model_path $workspace_full' >> /content/cronjob.sh
else:
  !echo 'bash $install_path"scripts/0_rsync.sh" $model_path "$workspace_full/model"' > /content/cronjob.sh





cron_min = "*"
cron_hour ="*"
if (Backup_hourly_on_gdrive):
  print("Time to backup: "+str(round((max_time-uptime)/3600))+" hours")
  cron_min = !date +"%M"
  cron_min = (int(cron_min[0])+5) %59
elif (round(max_time-3300-uptime) > 0):
  print("Time to backup: "+str(round((max_time-uptime)/3600))+" hours")
  hours_left = int(round(max_time-uptime)/3600)
  cron_hour = !(date +"%H")
  cron_hour = (int(cron_hour[0])+hours_left)%23
else:
  raise Exception("Session expires in less than an hour. Do a factory reset!")
  backup_cmd = ""

# cron_min = "*/5"
#save cronjob
!echo '$cron_min $cron_hour * * * root bash /content/cronjob.sh >> /content/cron_backup_log.txt' >> /etc/crontab
!sudo service cron start
crontab = !cat /etc/crontab
print(f"Cronjob entry: {crontab[-1]}")


cmd = f"DeepFaceLab/main.py train --training-data-src-dir {workspace_path}data_src/aligned --training-data-dst-dir {workspace_path}data_dst/aligned --pretraining-data-dir pretrain --model-dir {workspace_path}model --model "+Model

if Model == "Quick96":
  cmd+= " --pretrained-model-dir pretrain_Q96"

if Silent_Start:
  cmd+= " --silent-start"


train_cmd = cmd

!python $train_cmd

#Scripts

Run these scripts interactivly
Some may do not work, like xseg or interactive merger and training with preview

In [ ]:
#@title Run and select script, do not use for training!

#@markdown use arrow keys 
import ipywidgets as widgets

import ipywidgets as widgets
from os.path import join
from glob import glob
import os 

install_path = !cat /content/install_path
install_path = install_path[0]
script_dir = install_path+"/scripts/*.sh"

#audio_in = "eminem_drugs_cut.mp3"

scripts_arr = glob(script_dir)   
scripts_arr = sorted(list(map(os.path.basename, scripts_arr)))
if len(scripts_arr) == 0:
  #raise Exception("No scripts found")
  pass
script_widget = widgets.Select(options=scripts_arr, description="Select script", layout={'height':"250px", 'width': 'max-content'})

display(script_widget)

In [ ]:
!#@title run script

#@markdown Click Runtime->Inerrupt Execution (somtime multiple times) to stop
import os 
%cd /content
install_path = !cat /content/install_path
install_path = install_path[0]
script_dir = os.path.join(install_path, "scripts")
%cd $script_dir
script = script_widget.value
print(f"Running script {script}")
!sudo bash $script


# Tools


##Shell

In [ ]:
#@title build shell
from IPython.display import JSON
from google.colab import output
from subprocess import getoutput
import os

def shell(command):
  if command.startswith('cd'):
    path = command.strip().split(maxsplit=1)[1]
    os.chdir(path)
    return JSON([''])
  return JSON([getoutput(command)])
output.register_callback('shell', shell)

In [ ]:
#@title Run simple Shell
%%html
<div id=term_demo></div>
<script src="https://code.jquery.com/jquery-latest.js"></script>
<script src="https://cdn.jsdelivr.net/npm/jquery.terminal/js/jquery.terminal.min.js"></script>
<link href="https://cdn.jsdelivr.net/npm/jquery.terminal/css/jquery.terminal.min.css" rel="stylesheet"/>
<script>
  $('#term_demo').terminal(async function(command) {
      if (command !== '') {
          try {
              let res = await google.colab.kernel.invokeFunction('shell', [command])
              let out = res.data['application/json'][0]
              this.echo(new String(out))
          } catch(e) {
              this.error(new String(e));
          }
      } else {
          this.echo('');
      }
  }, {
      greetings: 'Welcome to Colab Shell',
      name: 'colab_demo',
      height: 250,
      prompt: 'colab > '
  });